In [61]:
# Reduction via PCA, ICA, and Double-DBM

In [62]:
# Cluster via K Means, DBSCAN, Spectral Clustering

In [63]:
# Graph and save V-measure, ARI, Adjusted Mutual Information

In [171]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
import csv
import pandas as pd
from pandas import DataFrame
def load_data(file_name: str = "Data.csv" , debug: bool = False ) -> DataFrame:
    
    try:
        with open(file_name, mode ='r')as file:
          csv_file = csv.reader(file)
        df = pd.read_csv(file_name)
        if debug:
            print("Dataframe after loading data")
            print(df.head(5))
    except:
        print("Could Not Load File")
   
    if debug:
        print("Loading Data Completed")
    return df

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Vectorizing Method: 
def vectorize_data(df, v_method: str = "TfidfVectorizer", debug:bool = False) -> DataFrame:
    try:
        all_vectorized_dfs = []

        # Iterate over each column except the first one
        for column in df.columns[3:]:
            if v_method == "TfidfVectorizer":
                vectorizer = TfidfVectorizer(stop_words='english')
                transformed_data = vectorizer.fit_transform(df[column])

                if debug:
                    print(f"Shape after vectorizing column '{column}':", transformed_data.shape)

                # Convert the sparse matrix to a DataFrame
                vectorized_df = pd.DataFrame(transformed_data.toarray(), columns=[f"{column}_{feature}" for feature in vectorizer.get_feature_names_out()])
                all_vectorized_dfs.append(vectorized_df)
            else:
                raise ValueError(f"Vectorizing method '{v_method}' is not supported.")

        # Concatenate all the vectorized DataFrames along the columns
        df_vectorized = pd.concat(all_vectorized_dfs, axis=1)

        # Append the first column of the original DataFrame
        df_final = pd.concat([df.iloc[:, [0]], df_vectorized], axis=1)

        if debug:
            print("Final DataFrame shape after vectorizing all columns:", df_final.shape)

        return df_final

    except Exception as e:
        print(f"Error while Vectorizing Data: {e}")
        return df


In [182]:
def __main__(file_name: str = "data.csv",debug: bool = False):
    df = load_data(file_name,False)
    df = vectorize_data(df,"TfidfVectorizer",debug)
debug = True
__main__("Smash 64 v1.csv",debug)

Shape after vectorizing column 'Quick Summary': (12, 586)
Final DataFrame shape after vectorizing all columns: (12, 587)
